In [ ]:
%run network-init.ipynb

# Get some training data

In [ ]:
matches = pd.read_csv(str(utils.datadir / "links" / "matches.csv"), sep="|")

In [ ]:
matches

In [ ]:
df = pd.read_pickle("dataframe.pickled")

In [ ]:
df.dropna(subset=("Fødeår",), inplace=True)

In [ ]:
df.set_index(["FT", "Kipnr", "Løbenr"], inplace=True)

In [ ]:
df.sort_index(inplace=True)

Getting raw data for all matches:

In [ ]:
def to_scored_frame(d):
    lots = []
    for t in d.itertuples():
        try:
            # TODO: optimize access
            s = df.loc[[t[1:4], t[4:]]]
            a = s.iloc[0]
            b = s.iloc[1]
        except KeyError:
            continue
        lots.append(score(a,b) + (True,))
    return pd.DataFrame(lots, columns=score_columns + ["label"])

In [ ]:
positive = utils.parallelize(matches[:1000], to_scored_frame)

Run through some arbitrary non-matches

In [ ]:
import random
import itertools

In [ ]:
years = [1845, 1850, 1860, 1880, 1885]

In [ ]:
ssize = 100
def generate_nonmatch_scores(_):
    lots = []
    s = df.loc[random.choice(years)].sample(ssize)
    for i in range(ssize-1):
        a = s.iloc[i]
        b = s.iloc[i+1]
        lots.append(score(a,b) + (False,))
    return lots

with multiprocessing.Pool() as p:
    res = p.map(generate_nonmatch_scores, [None for i in range(multiprocessing.cpu_count())])

In [ ]:
negative = pd.DataFrame(list(itertools.chain(*res)), columns=score_columns + ["label"])

In [ ]:
del(res)

In [ ]:
df_train = pd.concat([positive, negative])

In [ ]:
df_train

# Train and evaluate models

## Linear model

In [ ]:
input_fn_train = tf.estimator.inputs.pandas_input_fn(
    x=df_train,
    y=df_train.label,
    shuffle=True)

In [ ]:
model1.train(input_fn=input_fn_train)

In [ ]:
model1.evaluate(input_fn=input_fn_train)

## DNN model

In [ ]:
model2.train(input_fn_train)

In [ ]:
model2.evaluate(input_fn_train)